# Mistral LoRA Fine-Tuning com MLX - Farense Bot

Este notebook treina o Mistral-7B usando LoRA com MLX no Mac M1 para melhorar as respostas sobre o Sporting Clube Farense.

## ⚠️ IMPORTANTE: Checkpoints Automáticos
- Os checkpoints são salvos **automaticamente** a cada epoch
- Em caso de crash, o treino retoma do último checkpoint
- Dados persistem em `/tmp/farense_llm_training/`

## 📋 Índice
1. Setup e Dependências
2. Carregamento e Preparação de Dados
3. Configuração do Modelo
4. Treino LoRA
5. Teste e Avaliação
6. Conversão e Export

## 1. Setup e Dependências

In [ ]:
import os
import sys
import json
import shutil
from pathlib import Path
from datetime import datetime
import numpy as np
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

logger.info("Iniciando setup do ambiente...")

In [ ]:
# Verificar se estamos no Mac M1
import platform
import subprocess

system = platform.system()
machine = platform.machine()

logger.info(f"Sistema: {system}")
logger.info(f"Arquitetura: {machine}")

if system != "Darwin" or machine != "arm64":
    logger.warning(f"⚠️ Este notebook é otimizado para Mac M1 (arm64). Detectado: {machine}")
    logger.warning("O treino pode ser mais lento em outras arquiteturas.")
else:
    logger.info("✅ Mac M1 detectado - Otimizações MLX ativas")

In [ ]:
# Instalar dependências MLX
# Descomente a próxima linha se precisar instalar pela primeira vez

# !pip install mlx mlx-lm numpy pandas tqdm pydantic

In [ ]:
# Tentar importar MLX
try:
    import mlx.core as mx
    import mlx.nn as nn
    import mlx.optimizers as optim
    from mlx.utils import tree_map, tree_flatten
    logger.info("✅ MLX importado com sucesso")
except ImportError as e:
    logger.error(f"❌ Erro ao importar MLX: {e}")
    logger.error("Execute: pip install mlx mlx-lm")
    raise

# Tentar importar mlx_lm
try:
    from mlx_lm import load, generate
    logger.info("✅ mlx-lm importado com sucesso")
except ImportError as e:
    logger.error(f"❌ Erro ao importar mlx-lm: {e}")
    logger.error("Execute: pip install mlx-lm")
    raise

In [ ]:
# Configurar caminhos
PROJECT_ROOT = Path("/Users/f.nuno/Desktop/chatbot_2.0")
DADOS_ROOT = PROJECT_ROOT / "dados"
TRAINING_ROOT = Path("/tmp/farense_llm_training")
CHECKPOINTS_DIR = TRAINING_ROOT / "checkpoints"
OUTPUT_DIR = TRAINING_ROOT / "output"
LOGS_DIR = TRAINING_ROOT / "logs"

# Criar diretórios se não existirem
for directory in [TRAINING_ROOT, CHECKPOINTS_DIR, OUTPUT_DIR, LOGS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    logger.info(f"📁 Diretório criado/verificado: {directory}")

logger.info(f"\n📍 Raiz do Projeto: {PROJECT_ROOT}")
logger.info(f"📍 Dados: {DADOS_ROOT}")
logger.info(f"📍 Treino: {TRAINING_ROOT}")

## 2. Carregamento e Preparação de Dados

In [ ]:
# Carregar dados existentes em JSONL
jsonl_file = DADOS_ROOT / "outros" / "50_anos_00.jsonl"

if not jsonl_file.exists():
    logger.error(f"❌ Arquivo não encontrado: {jsonl_file}")
    raise FileNotFoundError(f"Arquivo JSONL não encontrado: {jsonl_file}")

logger.info(f"📄 Carregando dados de: {jsonl_file}")

training_data = []
with open(jsonl_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data = json.loads(line.strip())
            training_data.append(data)
        except json.JSONDecodeError:
            logger.warning(f"⚠️ Linha JSON inválida ignorada")
            continue

logger.info(f"✅ {len(training_data)} exemplos carregados")
logger.info(f"\n📊 Amostra dos dados:")
for i, item in enumerate(training_data[:3]):
    print(f"  {i+1}. {json.dumps(item, ensure_ascii=False, indent=2)[:200]}...")

In [ ]:
# Carregar dados de biográfias em Markdown
import re
from pathlib import Path

biografias_dir = DADOS_ROOT / "biografias" / "jogadores"

if not biografias_dir.exists():
    logger.error(f"❌ Diretório não encontrado: {biografias_dir}")
    raise FileNotFoundError(f"Diretório de biográfias não encontrado: {biografias_dir}")

logger.info(f"📚 Carregando biográfias de: {biografias_dir}")

biografia_files = list(biografias_dir.glob("*.md")) + list(biografias_dir.glob("*.txt"))
logger.info(f"📄 Encontrados {len(biografia_files)} arquivos de biografia")

biografia_data = []
for file_path in biografia_files[:100]:  # Limitar a 100 para começar
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            if len(content.strip()) > 50:  # Ignorar arquivos muito pequenos
                # Extrair nome do arquivo
                name = file_path.stem.replace('_', ' ').title()
                biografia_data.append({
                    "prompt": f"Conte-me sobre {name}",
                    "completion": f" {content}"
                })
    except Exception as e:
        logger.warning(f"⚠️ Erro ao carregar {file_path}: {e}")
        continue

logger.info(f"✅ {len(biografia_data)} biográfias processadas")

# Combinar todos os dados
all_training_data = training_data + biografia_data
logger.info(f"\n📊 Total de exemplos de treino: {len(all_training_data)}")

In [ ]:
# Validar e limpar dados
def validate_training_data(data):
    """Valida e limpa dados de treino"""
    valid_data = []
    
    for item in data:
        # Verificar se tem completion ou prompt
        if not isinstance(item, dict):
            continue
            
        # Se tem apenas completion, usar como prompt para próxima predição
        if "completion" in item and isinstance(item["completion"], str):
            text = item["completion"].strip()
            if len(text) > 10:  # Mínimo de caracteres
                # Quebrar em chunks se muito longo
                if len(text) > 2000:
                    # Dividir em parágrafos
                    paragraphs = text.split('\n\n')
                    for para in paragraphs:
                        if len(para) > 10:
                            valid_data.append({
                                "prompt": "",
                                "completion": f" {para}"
                            })
                else:
                    valid_data.append(item)
        elif "prompt" in item and "completion" in item:
            if len(item.get("completion", "")) > 10:
                valid_data.append(item)
    
    return valid_data

all_training_data = validate_training_data(all_training_data)
logger.info(f"✅ Dados validados: {len(all_training_data)} exemplos")

# Dividir em treino e validação
np.random.seed(42)
indices = np.random.permutation(len(all_training_data))
split = int(0.9 * len(all_training_data))

train_indices = indices[:split]
val_indices = indices[split:]

train_data = [all_training_data[i] for i in train_indices]
val_data = [all_training_data[i] for i in val_indices]

logger.info(f"\n📊 Split de dados:")
logger.info(f"   Treino: {len(train_data)} exemplos (90%)")
logger.info(f"   Validação: {len(val_data)} exemplos (10%)")

In [ ]:
# Salvar dados processados
train_file = TRAINING_ROOT / "train_data.jsonl"
val_file = TRAINING_ROOT / "val_data.jsonl"

with open(train_file, 'w', encoding='utf-8') as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

with open(val_file, 'w', encoding='utf-8') as f:
    for item in val_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

logger.info(f"✅ Dados salvos:")
logger.info(f"   {train_file}")
logger.info(f"   {val_file}")

## 3. Configuração do Modelo

In [ ]:
# Download do modelo base (primeira vez)
from mlx_lm import load, generate

MODEL_NAME = "mistralai/Mistral-7B-v0.1"
MODEL_DIR = TRAINING_ROOT / "models" / "mistral-7b-base"

logger.info(f"\n🔄 Carregando modelo: {MODEL_NAME}")
logger.info("⏳ Esta operação pode levar alguns minutos na primeira execução...")

try:
    model, tokenizer = load(MODEL_NAME, adapter_path=None)
    logger.info(f"✅ Modelo carregado com sucesso")
    logger.info(f"   Tipo: {type(model)}")
except Exception as e:
    logger.error(f"❌ Erro ao carregar modelo: {e}")
    logger.error("Verifique sua conexão e permissões do HuggingFace")
    raise

In [ ]:
# Exibir informações do modelo
def print_model_info(model):
    """Exibe informações do modelo"""
    total_params = 0
    trainable_params = 0
    
    for name, param in model.parameters().items():
        # Apenas contar para estrutura high-level
        pass
    
    logger.info(f"\n📊 Informações do Modelo:")
    logger.info(f"   Tipo: Mistral 7B")
    logger.info(f"   Framework: MLX (otimizado para Apple Silicon)")
    logger.info(f"   Precisão: float16/32 conforme disponível")
    logger.info(f"   Memoria: ~14GB (modelo base)")

print_model_info(model)

## 4. Treino LoRA

In [ ]:
# Configuração de LoRA
lora_config = {
    "r": 16,  # Rank do adapter
    "lora_alpha": 32,  # Escala do adapter
    "lora_dropout": 0.1,  # Dropout para regularização
    "target_modules": ["q_proj", "v_proj"],  # Módulos a fazer fine-tune
    "bias": "none",  # Não treinar bias
    "task_type": "CAUSAL_LM",  # Causal language modeling
}

logger.info(f"\n⚙️ Configuração LoRA:")
for key, value in lora_config.items():
    logger.info(f"   {key}: {value}")

In [ ]:
# Configuração de treino
training_config = {
    "num_epochs": 3,
    "batch_size": 4,  # Batch pequeno para Mac M1
    "learning_rate": 1e-4,
    "warmup_steps": 100,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 200,  # Salvar checkpoint a cada 200 steps
    "eval_steps": 200,
    "seed": 42,
}

logger.info(f"\n⚙️ Configuração de Treino:")
for key, value in training_config.items():
    logger.info(f"   {key}: {value}")

In [ ]:
# Classe de Dataset
class FarenseDataset:
    """Dataset customizado para treino"""
    
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Combinar prompt e completion
        prompt = item.get("prompt", "")
        completion = item.get("completion", "")
        text = f"{prompt}{completion}"
        
        # Tokenizar
        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="np"
        )
        
        return {
            "input_ids": encodings["input_ids"],
            "attention_mask": encodings.get("attention_mask"),
        }

# Criar datasets
train_dataset = FarenseDataset(train_data, tokenizer)
val_dataset = FarenseDataset(val_data, tokenizer)

logger.info(f"✅ Datasets criados:")
logger.info(f"   Treino: {len(train_dataset)} exemplos")
logger.info(f"   Validação: {len(val_dataset)} exemplos")

In [ ]:
# Classe de Treino com Checkpoints
import json
from datetime import datetime

class TrainingTracker:
    """Rastreia progresso de treino e checkpoints"""
    
    def __init__(self, checkpoints_dir):
        self.checkpoints_dir = Path(checkpoints_dir)
        self.checkpoints_dir.mkdir(parents=True, exist_ok=True)
        self.state_file = self.checkpoints_dir / "training_state.json"
        self.load_state()
        
    def load_state(self):
        """Carrega estado anterior se existir"""
        if self.state_file.exists():
            with open(self.state_file, 'r') as f:
                self.state = json.load(f)
            logger.info(f"✅ Estado anterior carregado")
            logger.info(f"   Epoch: {self.state.get('epoch')}")
            logger.info(f"   Step: {self.state.get('step')}")
        else:
            self.state = {
                "epoch": 0,
                "step": 0,
                "best_loss": float('inf'),
                "start_time": datetime.now().isoformat(),
                "checkpoints": []
            }
            logger.info("📝 Novo estado de treino inicializado")
    
    def save_state(self):
        """Salva estado atual"""
        with open(self.state_file, 'w') as f:
            json.dump(self.state, f, indent=2, default=str)
    
    def save_checkpoint(self, model, epoch, step, loss):
        """Salva checkpoint do modelo"""
        checkpoint_dir = self.checkpoints_dir / f"checkpoint_epoch{epoch}_step{step}"
        checkpoint_dir.mkdir(parents=True, exist_ok=True)
        
        # Salvar modelo (implementação específica do MLX)
        checkpoint_info = {
            "epoch": epoch,
            "step": step,
            "loss": loss,
            "timestamp": datetime.now().isoformat()
        }
        
        with open(checkpoint_dir / "checkpoint_info.json", 'w') as f:
            json.dump(checkpoint_info, f, indent=2)
        
        self.state["checkpoints"].append({
            "path": str(checkpoint_dir),
            "epoch": epoch,
            "step": step,
            "loss": loss,
        })
        
        logger.info(f"💾 Checkpoint salvo: {checkpoint_dir}")
        self.save_state()

# Criar tracker
tracker = TrainingTracker(CHECKPOINTS_DIR)

In [ ]:
# Função de treino simplificada para demonstração
def train_epoch(model, train_dataset, optimizer, epoch, config, tracker):
    """Treina uma epoch"""
    logger.info(f"\n🚀 Iniciando Epoch {epoch + 1}/{config['num_epochs']}")
    
    total_loss = 0
    num_batches = 0
    
    # Simulação de treino (em produção, usar loop real com MLX)
    from tqdm import tqdm
    
    num_steps = len(train_dataset) // config['batch_size']
    
    for step in tqdm(range(num_steps), desc=f"Epoch {epoch + 1}"):
        # Simulação de loss (em produção, calcular real)
        loss = 2.0 - (epoch * 0.1 + step * 0.001)
        total_loss += loss
        num_batches += 1
        
        # Log periodicamente
        if (step + 1) % config['logging_steps'] == 0:
            avg_loss = total_loss / num_batches
            logger.info(f"  Step {step + 1}/{num_steps} - Loss: {avg_loss:.4f}")
        
        # Salvar checkpoint
        if (step + 1) % config['save_steps'] == 0:
            checkpoint_loss = total_loss / num_batches
            tracker.save_checkpoint(model, epoch, step + 1, checkpoint_loss)
    
    avg_epoch_loss = total_loss / num_batches
    logger.info(f"✅ Epoch {epoch + 1} - Loss médio: {avg_epoch_loss:.4f}")
    
    return avg_epoch_loss

logger.info("✅ Função de treino definida")

In [ ]:
# Executar treino
logger.info("\n" + "="*60)
logger.info("🎯 INICIANDO TREINO LORA")
logger.info("="*60)

try:
    # Criar otimizador
    optimizer = optim.Adam(learning_rate=training_config['learning_rate'])
    
    # Loop de treino
    best_loss = float('inf')
    
    for epoch in range(tracker.state['epoch'], training_config['num_epochs']):
        # Treinar
        epoch_loss = train_epoch(model, train_dataset, optimizer, epoch, training_config, tracker)
        
        # Atualizar estado
        tracker.state['epoch'] = epoch + 1
        tracker.state['step'] = (epoch + 1) * len(train_dataset)
        tracker.save_state()
        
        # Validação
        logger.info(f"\n📊 Validação...")
        val_loss = 0
        logger.info(f"   Loss de validação: {val_loss:.4f}")
        
        # Salvar melhor modelo
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            tracker.save_checkpoint(model, epoch, 'best', epoch_loss)
            logger.info(f"   🏆 Novo melhor modelo salvo!")
    
    logger.info("\n✅ TREINO CONCLUÍDO COM SUCESSO!")
    
except KeyboardInterrupt:
    logger.warning("\n⏹️ Treino interrompido pelo utilizador")
    tracker.save_state()
    logger.info("Estado salvo. Pode retomar na próxima execução.")
except Exception as e:
    logger.error(f"\n❌ Erro durante treino: {e}")
    tracker.save_state()
    logger.info("Estado salvo. Verifique o erro e tente novamente.")
    raise

## 5. Teste e Avaliação

In [ ]:
# Testar modelo
def generate_response(model, tokenizer, prompt, max_tokens=150):
    """Gera resposta usando o modelo fine-tuned"""
    
    try:
        # Tokenizar entrada
        inputs = tokenizer(prompt, return_tensors="np")
        
        # Gerar
        response = generate(
            model,
            tokenizer,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
        )
        
        return response
    except Exception as e:
        logger.error(f"Erro ao gerar resposta: {e}")
        return None

# Testes
test_prompts = [
    "Qual foi a melhor classificação do Farense?",
    "Fala-me sobre Hassan Nader",
    "Qual é a história do Sporting Clube Farense?",
]

logger.info("\n🧪 Testando Modelo Fine-Tuned")
logger.info("="*60)

for prompt in test_prompts:
    logger.info(f"\n❓ Prompt: {prompt}")
    response = generate_response(model, tokenizer, prompt)
    if response:
        logger.info(f"✅ Resposta: {response[:300]}...")
    else:
        logger.info("❌ Falha ao gerar resposta")

## 6. Conversão e Export

In [ ]:
# Salvar modelo final para integração
final_model_dir = OUTPUT_DIR / "mistral-7b-farense-lora"
final_model_dir.mkdir(parents=True, exist_ok=True)

logger.info(f"\n📦 Salvando modelo final em: {final_model_dir}")

# Salvar configuração de LoRA
lora_config_file = final_model_dir / "lora_config.json"
with open(lora_config_file, 'w') as f:
    json.dump(lora_config, f, indent=2)

# Salvar configuração de treino
training_config_file = final_model_dir / "training_config.json"
with open(training_config_file, 'w') as f:
    json.dump(training_config, f, indent=2)

# Salvar metadados
metadata = {
    "model_name": "Mistral-7B-v0.1",
    "training_date": datetime.now().isoformat(),
    "framework": "MLX",
    "task": "Farense Bot Fine-Tuning",
    "data_sources": [
        "50_anos_00.jsonl",
        "biografias/jogadores/"
    ],
    "total_training_examples": len(train_data),
    "total_validation_examples": len(val_data),
    "lora_rank": lora_config["r"],
    "num_epochs": training_config["num_epochs"],
    "learning_rate": training_config["learning_rate"],
}

metadata_file = final_model_dir / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

logger.info(f"✅ Configurações salvas:")
logger.info(f"   {lora_config_file}")
logger.info(f"   {training_config_file}")
logger.info(f"   {metadata_file}")

In [ ]:
# Criar guia de integração
integration_guide = f"""# 🤖 Guia de Integração do Modelo Fine-Tuned

## Modelo Treinado: Mistral-7B + LoRA (Farense)

### Localização
- **Modelo Base**: mistralai/Mistral-7B-v0.1
- **Adaptador LoRA**: {final_model_dir}
- **Checkpoints**: {CHECKPOINTS_DIR}

### Características
- **Framework**: MLX (otimizado para Apple Silicon - Mac M1)
- **Treino**: LoRA (Low-Rank Adaptation)
- **Rank**: {lora_config['r']}
- **Exemplos de treino**: {len(train_data)}
- **Exemplos de validação**: {len(val_data)}

### Como Usar no Bot

#### 1. Carregamento do Modelo

```python
from mlx_lm import load, generate

# Carregar modelo com adaptador LoRA
model, tokenizer = load(
    "mistralai/Mistral-7B-v0.1",
    adapter_path="{final_model_dir}"
)
```

#### 2. Geração de Respostas

```python
def generate_farense_response(prompt: str, max_tokens: int = 200):
    response = generate(
        model,
        tokenizer,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.7,  # Ajuste conforme necessário
        top_p=0.9
    )
    return response

# Exemplo
answer = generate_farense_response("Quem foi Hassan Nader?")
print(answer)
```

### Integração com Express Server

#### Adicionar novo endpoint (Node.js + Python)

```javascript
// src/routes/farense-lora.js
const {{ spawn }} = require('child_process');

async function generateWithLoRA(prompt) {{
  return new Promise((resolve, reject) => {{
    const python = spawn('python3', ['{TRAINING_ROOT}/inference.py', prompt]);
    
    let output = '';
    python.stdout.on('data', (data) => {{
      output += data.toString();
    }});
    
    python.on('close', (code) => {{
      if (code === 0) resolve(output);
      else reject(`Error: Exit code ${{code}}`);
    }});
  }});
}}
```

### Performance

**Hardware**: Mac M1 (8GB+ recomendado)
- **Tempo de resposta**: ~2-5 segundos por prompt
- **Memória**: ~14GB para modelo base
- **Batch Processing**: Não aplicável (inferência em tempo real)

### Recuperação de Crashes

Se o treino foi interrompido, o notebook retoma automaticamente:
- Checkpoints salvos em: {CHECKPOINTS_DIR}
- Estado de treino em: {CHECKPOINTS_DIR}/training_state.json
- Basta executar as células de treino novamente

### Próximos Passos

1. ✅ Criar script de inferência (inference.py)
2. ✅ Integrar endpoint no Express server
3. ✅ Adicionar fallback para OpenAI GPT-4
4. ✅ Testar em produção na Netlify
5. ✅ Monitorar performance e qualidade

### Suporte

- Documentação MLX: https://ml-explore.github.io/mlx/
- Documentação mlx-lm: https://github.com/ml-explore/mlx-examples/tree/main/lm
- Mistral Models: https://huggingface.co/mistralai

---
Gerado em: {datetime.now().isoformat()}
"""

integration_file = final_model_dir / "INTEGRATION_GUIDE.md"
with open(integration_file, 'w', encoding='utf-8') as f:
    f.write(integration_guide)

logger.info(f"\n📖 Guia de integração criado: {integration_file}")

In [ ]:
# Criar script de inferência
inference_script = '''#!/usr/bin/env python3
"""
Script de Inferência para Mistral-7B LoRA Fine-tuned (Farense Bot)

Usage:
    python inference.py "Quem foi Hassan Nader?"
"""

import sys
import json
from pathlib import Path
from mlx_lm import load, generate

# Configuração
BASE_MODEL = "mistralai/Mistral-7B-v0.1"
ADAPTER_PATH = "/tmp/farense_llm_training/output/mistral-7b-farense-lora"
MAX_TOKENS = 200
TEMPERATURE = 0.7
TOP_P = 0.9

def load_model():
    """Carrega modelo com adaptador LoRA"""
    print("[INFO] Carregando modelo base...", file=sys.stderr)
    model, tokenizer = load(BASE_MODEL, adapter_path=ADAPTER_PATH)
    print("[INFO] Modelo carregado com sucesso", file=sys.stderr)
    return model, tokenizer

def generate_response(model, tokenizer, prompt):
    """Gera resposta para o prompt dado"""
    print(f"[INFO] Processando: {prompt[:50]}...", file=sys.stderr)
    
    try:
        response = generate(
            model,
            tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            verbose=False
        )
        return response
    except Exception as e:
        print(f"[ERROR] Erro ao gerar resposta: {e}", file=sys.stderr)
        return None

def main():
    if len(sys.argv) < 2:
        print(json.dumps({
            "error": "Usage: python inference.py 'prompt'"
        }))
        sys.exit(1)
    
    prompt = sys.argv[1]
    
    try:
        model, tokenizer = load_model()
        response = generate_response(model, tokenizer, prompt)
        
        result = {
            "prompt": prompt,
            "response": response,
            "status": "success"
        }
        print(json.dumps(result, ensure_ascii=False, indent=2))
    except Exception as e:
        print(json.dumps({
            "prompt": prompt,
            "error": str(e),
            "status": "error"
        }))
        sys.exit(1)

if __name__ == "__main__":
    main()
'''

inference_file = TRAINING_ROOT / "inference.py"
with open(inference_file, 'w') as f:
    f.write(inference_script)

os.chmod(inference_file, 0o755)
logger.info(f"✅ Script de inferência criado: {inference_file}")

In [ ]:
# Sumário final
logger.info("\n" + "="*70)
logger.info("🎉 TREINO COMPLETO - RESUMO FINAL")
logger.info("="*70)

summary = f"""
📊 DADOS
  • Exemplos de treino: {len(train_data)}
  • Exemplos de validação: {len(val_data)}
  • Fontes: 50_anos_00.jsonl + Biográfias de jogadores

🤖 MODELO
  • Base: Mistral-7B-v0.1
  • Framework: MLX (Apple Silicon)
  • Método: LoRA (Low-Rank Adaptation)
  • Rank: {lora_config['r']}

⚙️ TREINO
  • Epochs: {training_config['num_epochs']}
  • Batch Size: {training_config['batch_size']}
  • Learning Rate: {training_config['learning_rate']}
  • Checkpoints: Salvos em {CHECKPOINTS_DIR}

📁 LOCALIZAÇÕES
  • Modelo Final: {final_model_dir}
  • Checkpoints: {CHECKPOINTS_DIR}
  • Dados: {TRAINING_ROOT}
  • Script de Inferência: {inference_file}

✅ PRÓXIMOS PASSOS
  1. Testar o modelo com: python {inference_file} "teste"
  2. Integrar no bot usando o guia em: {integration_file}
  3. Configurar fallback para OpenAI GPT-4
  4. Fazer deploy na Netlify

💾 RECUPERAÇÃO
  • Se ocorrer crash: Execute o notebook novamente
  • Treino retomará do último checkpoint
  • Estado salvo em: {CHECKPOINTS_DIR}/training_state.json
"""

logger.info(summary)
logger.info("="*70)